# Download and transfer image data

This notebook gives some methods for getting image data to a VM for running PixPlot analysis.

It also shows how to sync that data to a local machine, for running demos and looking through image folders.

## Download images from Google Cloud Storage

The gsutil URIs for the dataset are of the form:

```gs://hathitrust-full_1800-50/chi/000/chi.085041031_00000006_00.jpg```

The bucket name is **hathitrust-full_1800-50** and the storage class is Nearline (multiple zones). The simplest way to land this data to a file is with gsutil, from the VM. But Python bindings are available as well.

In [13]:
!ls /mnt/c/users/stephen-krewson/documents/_datasets

cnn-models
dhlab-workshop
early-19C-illustrations_full-index.ann
early-19C-illustrations_full-index_list.txt.gz
early-19C-illustrations_metadata.csv
keys.py
medical-heritage-library
munroe-francis
peter-parley
training_11-class-junk-filter.zip


In [8]:
import pandas as pd
import os

In [19]:
# The intended location for the data. If within Git, remember to not version .jpg files.
# Make sure the data is being backed up on my 4TB disk; so best to keep it on C:\
DEST = "/mnt/c/users/stephen-krewson/documents/_datasets/peter-parley/images"

In [20]:
!ls $DEST

In [21]:
# POSE THE QUESTION BEFORE RUNNING THE JOB! Consider adding "geography" to Peter Parley query?
# How many more volumes would this add?
# PixPlot 1: Educational publishers (Carter Hendee, Munroe Francis, etc.). Cf. Goodrich's output and memoirs.
# PixPlot 2: Geography books and Peter Parley texts. Searching title for geography OR peter parley. To avoid dedup step.